In [3]:
import tensorflow as tf
import os

In [ ]:
# file paths
base_dir = "../data/tf_records"
splits = ["bal_train", "eval", "unbal_train"]

In [ ]:
# tfrecord files list
for split in splits:
    split_dir = os.path.join(base_dir, split)
    tfrecord_files = [os.path.join(split_dir, f) for f in os.listdir(split_dir) if f.endswith('.tfrecord')]

In [26]:
def inspect_tfrecord_n(file_path, n):
    raw_dataset = tf.data.TFRecordDataset(file_path)
    file_name = os.path.basename(file_path)
    
    for i, raw_record in enumerate(raw_dataset.take(n), start=1):
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        
        print(f"Features del registro numero {i} de {file_name}:")
        for key, feature in example.features.feature.items():
            if feature.HasField("bytes_list"):
                value = feature.bytes_list.value[0].decode('utf-8') if feature.bytes_list.value else ""
            elif feature.HasField("float_list"):
                value = feature.float_list.value[0] if feature.float_list.value else 0.0
            elif feature.HasField("int64_list"):
                value = feature.int64_list.value[0] if feature.int64_list.value else 0
            else:
                value = "Unknown Type"
            print(f'"{key}": {value}')
        print('')

We inspect an arbitrary record from the tfrecord list

In [30]:
file_path = tfrecord_files[10]
inspect_tfrecord_n(file_path, 3)

Features del registro numero 1 de -9.tfrecord:
"end_time_seconds": 40.0
"video_id": -9wBfK2qA1Q
"start_time_seconds": 30.0
"labels": 31

Features del registro numero 2 de -9.tfrecord:
"end_time_seconds": 40.0
"video_id": -9ICHzifGqM
"start_time_seconds": 30.0
"labels": 137

Features del registro numero 3 de -9.tfrecord:
"end_time_seconds": 40.0
"video_id": -9zxpWMQxL4
"start_time_seconds": 30.0
"labels": 468

